In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [3]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [5]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))
    


In [16]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], 
                                 dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)


# Call the figure to plot the data.
fig


Figure(layout=FigureLayout(height='420px'))

In [17]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [18]:
# Heatmap of percent clouds
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [19]:
# Heatmap of wind speed
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [26]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 85


In [27]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
6,6,Butaritari,KI,2021-10-15 21:17:49,3.0707,172.7902,82.69,70,99,11.16
17,17,Harper,LR,2021-10-15 21:17:55,4.3750,-7.7169,78.76,87,91,8.23
18,18,Tam Ky,VN,2021-10-15 21:17:55,15.5667,108.4833,76.98,95,100,4.88
22,22,Cayenne,GF,2021-10-15 21:15:04,4.9333,-52.3333,83.08,84,100,1.01
23,23,Kavieng,PG,2021-10-15 21:17:58,-2.5744,150.7967,83.68,74,86,11.34
24,24,Sao Filipe,CV,2021-10-15 21:17:58,14.8961,-24.4956,78.87,76,40,10.58
27,27,Vaini,TO,2021-10-15 21:17:59,-21.2000,-175.2000,78.96,89,40,5.75
29,29,Piacabucu,BR,2021-10-15 21:18:00,-10.4056,-36.4344,77.76,77,93,9.15
30,30,Hilo,US,2021-10-15 21:18:01,19.7297,-155.0900,82.27,80,20,13.80
34,34,Bac Lieu,VN,2021-10-15 21:18:04,9.2850,105.7244,78.37,86,95,3.33


In [28]:
preferred_cities_df.count()

City_ID       171
City          171
Country       171
Date          171
Lat           171
Lng           171
Max Temp      171
Humidity      171
Cloudiness    171
Wind Speed    171
dtype: int64

In [29]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
6,Butaritari,KI,82.69,3.0707,172.7902,
17,Harper,LR,78.76,4.3750,-7.7169,
18,Tam Ky,VN,76.98,15.5667,108.4833,
22,Cayenne,GF,83.08,4.9333,-52.3333,
23,Kavieng,PG,83.68,-2.5744,150.7967,
24,Sao Filipe,CV,78.87,14.8961,-24.4956,
27,Vaini,TO,78.96,-21.2000,-175.2000,
29,Piacabucu,BR,77.76,-10.4056,-36.4344,
30,Hilo,US,82.27,19.7297,-155.0900,
34,Bac Lieu,VN,78.37,9.2850,105.7244,


In [30]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")


Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [31]:
hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Hotel Name
6,Butaritari,KI,82.69,3.0707,172.7902,Isles Sunset Lodge
17,Harper,LR,78.76,4.3750,-7.7169,Screensaver
18,Tam Ky,VN,76.98,15.5667,108.4833,Khách Sạn Đông Á
22,Cayenne,GF,83.08,4.9333,-52.3333,Hôtel Le Dronmi
23,Kavieng,PG,83.68,-2.5744,150.7967,Nusa Island Retreat


In [33]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [34]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [36]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))